<a href="https://colab.research.google.com/github/Alecia113/NLP-Ex/blob/main/L6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dHMM 什么是隐形：part of speech，什么是显性：单词

In [96]:
import nltk
import sys
nltk.download('brown')

from nltk.corpus import brown
from nltk.corpus import treebank
import nltk

# Downloading required corpus
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [97]:

brown_tags_words = []
brown_tagged_sents = brown.tagged_sents()

for sent in brown_tagged_sents:
    # sent is a list of word/tag pairs
    # add START/START at the beginning
    brown_tags_words.append( ("START", "START") )
    
    # then all the tag/word pairs for the word/tag pairs in the sentence.
    # shorten tags to 3 characters each
    brown_tags_words.extend([ (tag[:3], word) for (word, tag) in sent ])
    # then END/END
    brown_tags_words.append( ("END", "END") )

cfd_tagwords = nltk.ConditionalFreqDist(brown_tags_words) 
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)
brown_tags = [tag for (tag, word) in brown_tags_words ]
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)


In [98]:
#要的***

distinct_tags = set(brown_tags) #词性库
sentence =[
    word_tokenize("This race is awesome, I want to race too."),
    word_tokenize("That race is silly, I do not want to race.")
]
sentlen = len(sentence)
def test_model(test_samples):
  token_sequences = []
  for l in range(sentlen):
    token = []
    viterbi = [ ]
    backpointer = [ ]

    first_viterbi = { }
    first_backpointer = { }
    for tag in distinct_tags:

        if tag == "START": continue
        first_viterbi[ tag ] = cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[l][0] )
        first_backpointer[ tag ] = "START"
        
    viterbi.append(first_viterbi)
    backpointer.append(first_backpointer)

    currbest = max(first_viterbi.keys(), key = lambda tag: first_viterbi[ tag ])

    for wordindex in range(0, len(sentence[l])):
      this_viterbi = { }
      this_backpointer = { }
      prev_viterbi = viterbi[-1]


      for tag in distinct_tags:
          if tag == "START": continue
          for word in sentence[l]:  # P[Y]*P(X|Y)*P(w|X)
            best_previous = max(prev_viterbi.keys(),
                                key = lambda prevtag: prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * cpd_tagwords[tag].prob(sentence[l][wordindex]))
            this_viterbi[ tag ] = prev_viterbi[ best_previous] * cpd_tags[ best_previous ].prob(tag) * cpd_tagwords[ tag].prob(sentence[l][wordindex])
            this_backpointer[ tag ] = best_previous
      currbest = max(this_viterbi.keys(), key = lambda tag: this_viterbi[ tag ])
      token.append(currbest)
      viterbi.append(this_viterbi)
      backpointer.append(this_backpointer)
    token_sequences.append(token)
  return token_sequences
print(sentence)
print(test_model(sentence))

[['This', 'race', 'is', 'awesome', ',', 'I', 'want', 'to', 'race', 'too', '.'], ['That', 'race', 'is', 'silly', ',', 'I', 'do', 'not', 'want', 'to', 'race', '.']]
[['DT', 'NN', 'BEZ', 'JJ', ',', 'PPS', 'VB', 'TO', 'VB', 'QL', '.'], ['DT', 'NN', 'BEZ', 'JJ', ',', 'PPS', 'DO', '*', 'VB', 'TO', 'VB', '.']]


In [ ]:
# DT, NN, VBZ(BEZ),NN(JJ), , ,  PRP(PPS),VBP(VB),TO,VB,RB(OL),.; DT,NN,VBZ(BEZ),JJ, , , PRP(PPS), VBP(DO), RB(*),VB,TO,VB,./